# Simple Web Server

### What is a web server?
A web server is a server software that is capable of handling HTTP requests to produce HTTP responses. HTTP (hyper text transfer protocol) is a widely used protocol across the information to transfer web pages, documents (and other files) across different networks.

eg. Apache, nginx, a web server that you implement programatically

### Who is a web client?
A web client is a software application that is capable of sending HTTP requests to any web server and parsing the received response in a user-presentable form. (abstracting away the internal process)

eg. Chrome, Firefox, a web client that you implement programatically

### Static vs Dynamic Content
A web server is capable of sending static as well as dynamic responses.

Static documents residing inside the **webroot** of a web server computer are served as-is by the web server as a part of the HTTP response to its clients.

Dynamic resourcses are those resources which require some amount of programmatic parsing / scripting at the time of the receiving of the request such that each client request receives a response which is dynamically generated.

** A simple web server to serve static files. **

In [1]:
import http.server
import socketserver

TCP Port where the web server will run.

In [2]:
bindPort = 8080
bindHost = ''

In [3]:
handler = http.server.SimpleHTTPRequestHandler

**Webroot** for the web server.

In [4]:
!pwd

/Users/swg/Desktop


In [5]:
try:
    with socketserver.TCPServer((bindHost, bindPort), handler) as httpd:
        print('http server running at port', bindPort)
        httpd.serve_forever()
except KeyboardInterrupt:
    print('interrupted, server stopped.')

http server running at port 8080


127.0.0.1 - - [07/Oct/2018 16:54:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2018 16:54:23] "GET /Raspberry%20Pi/ HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2018 16:54:23] code 404, message File not found
127.0.0.1 - - [07/Oct/2018 16:54:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Oct/2018 16:54:25] "GET /Raspberry%20Pi/gpio.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2018 16:54:25] code 404, message File not found
127.0.0.1 - - [07/Oct/2018 16:54:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Oct/2018 16:54:33] "GET /Raspberry%20Pi/Circuit%20Diagram%201.PNG HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2018 16:54:33] code 404, message File not found
127.0.0.1 - - [07/Oct/2018 16:54:33] "GET /favicon.ico HTTP/1.1" 404 -


interrupted, server stopped.


#### Sample HTTP request

`GET /resourcename HTTP/1.1
Host: subdomain.anyhost.tld
Accept: */*
User-Agent: Mozilla/5.0
`

#### Sample HTTP Response

`HTTP/1.1 200 OK
Content-Type: text/plain
Content-Length: 27
Date: Fri, 12 Oct 2018 11:52:08 GMT
Connection: close`

`This is a sample text file.`

Same web server directly from the shell.

In [7]:
!python -m http.server 8080

Serving HTTP on 0.0.0.0 port 8080 (http://0.0.0.0:8080/) ...
127.0.0.1 - - [07/Oct/2018 17:41:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2018 17:41:45] "GET /Using%20Twitter%20Data%20for%20Research%20%28Plus%29.pdf HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2018 17:41:46] code 404, message File not found
127.0.0.1 - - [07/Oct/2018 17:41:46] "GET /favicon.ico HTTP/1.1" 404 -
^C

Keyboard interrupt received, exiting.


# WSGI (Web Server Gateway Interface)

## What is WSGI?
WSGI stands for Web Server Gateway Interface. Most commonly used web servers support this interface. It is not a framework or a module or a python package or an API, it is a specification for an interface that allows us to create wsgi applications that will run on any web server which supports that spec. It is also considered as an an improvement of its predecessor, Common Gateway Interface (CGI).

eg of servers supporting WSGI: Apache (with mod_wsgi), gunicorn, uWsgi, wsgiref, etc.

## WSGI Applications
Applications that conform to the WSGI specification and possess a callable function which is used to generate HTTP response (for web clients) on the basis of HTTP requests are called wsgi applications.

**Spec**:
1. must accept two arguments (a dictionary containing CGI like variables, a callback function that is used to trigger the HTTP response headers)
2. must return the HTTP body of the response to the server as an iterator of strings

In [1]:
def application(environ, startResponse):
    status = '200 OK'
    body = 'Hello World!'
    headers = [
        ('Content-Type', 'text/plain'),
        ('Content-Length', str(len(body)))
    ]
    startResponse(status, headers)
    data = body.encode('utf-8')
    return [data]

In [2]:
bindPort = 8080
bindHost = ''

Run the wsgi application using the python built-in wgsiref server.

In [3]:
from wsgiref.simple_server import make_server

In [4]:
httpd = make_server(bindHost, bindPort, application)

In [5]:
try:
    httpd.serve_forever()
except KeyboardInterrupt:
    print('interrupted, server stopped.')

127.0.0.1 - - [07/Oct/2018 18:49:26] "GET /myresource HTTP/1.1" 200 12
127.0.0.1 - - [07/Oct/2018 18:49:34] "GET / HTTP/1.1" 200 12
127.0.0.1 - - [07/Oct/2018 18:49:39] "GET / HTTP/1.1" 200 12
127.0.0.1 - - [07/Oct/2018 18:49:42] "GET / HTTP/1.1" 200 12
127.0.0.1 - - [07/Oct/2018 18:49:47] "GET / HTTP/1.1" 200 12
127.0.0.1 - - [07/Oct/2018 18:49:49] "GET /realtimefeed.dyn.html HTTP/1.1" 200 12
127.0.0.1 - - [07/Oct/2018 18:49:50] "GET /running HTTP/1.1" 200 12
127.0.0.1 - - [07/Oct/2018 18:49:50] "GET /favicon.ico HTTP/1.1" 200 12


interrupted, server stopped.


In [6]:
!pip install gunicorn

Running the wsgi application using the gunicorn web server instead. (Google App Engine environments use this web server as a default, so its both popular and powerful)

In [ ]:
# will run callable object `app` from main.py
!gunicorn -b :8080 main:app


### References:
1. https://docs.python.org/3/library/http.server.html
2. http://wsgi.tutorial.codepoint.net/
3. http://wsgi.tutorial.codepoint.net/application-interface
4. https://cloud.google.com/appengine/docs/standard/python3/testing-and-deploying-your-app
5. https://wsgi.readthedocs.io/en/latest/what.html
6. https://gunicorn.org
7. https://docs.python.org/2/library/simplehttpserver.html
8. https://docs.python.org/3.7/library/wsgiref.html

@selfLink: https://github.com/swghosh/UG-Courses/blob/master/Others/SimpleWebServer.ipynb